In [1]:
from selenium import webdriver
import pandas as pd
import time
import re
import os
import requests
from PIL import Image
from io import BytesIO

In [2]:
folder_name = 'books_posters'
# Make directory if it doesn't already exist
if not os.path.exists(folder_name):
    os.makedirs(folder_name)

In [3]:
driver = webdriver.Chrome(r"driver\chromedriver.exe")
driver.get("http://books.toscrape.com/catalogue/category/books_1/index.html")

In [4]:
def StarConversion(value):
    if value == "One":
        return 1
    elif value == "Two":
        return 2
    elif value == "Three":
        return 3
    elif value == "Four":
        return 4
    elif value == "Five":
        return 5 

In [6]:
all_details = []
book_num = 1
for c in range(1,51):
    try:
        # get the page
        driver.get("http://books.toscrape.com/catalogue/category/books_1/page-{}.html".format(c))
        print("\nPage link : http://books.toscrape.com/catalogue/category/books_1/page-{}.html".format(c))
        # find all books in the current page
        incategory = driver.find_elements_by_class_name("product_pod")
        
        # Generate a list of links for every book
        links = []
        for item in incategory:
            # get the href property (the link)
            a = item.find_element_by_tag_name("h3").find_element_by_tag_name("a").get_property("href")
            # Append the link to list links
            links.append(a)

        # Lets loop through each link to acces the page of each book
        books = 1
        print('Book num  : ', end='  ')
        for link in links:
            print(books, end='  ')
            books += 1
            # get one book url
            driver.get(link)
            # Title of the book
            title = driver.find_element_by_xpath("//*[@id='content_inner']/article/div[1]/div[2]/h1")
            # Price of the book
            price = driver.find_element_by_xpath("//*[@id='content_inner']/article/div[1]/div[2]/p[1]")
            # Stock - number of copies available for the book
            stock = driver.find_element_by_xpath("//*[@id='content_inner']/article/div[1]/div[2]/p[2]")
            # Stock comes as string so we need to use this regex to exract digits
            stock = int(re.findall("\d+",stock.text)[0])
            # Stars - Actual stars are values of class attribute
            stars = driver.find_element_by_xpath("//*[@id='content_inner']/article/div[1]/div[2]/p[3]").get_attribute("class")
            # convert string to number. Stars are like One, Two, Three ... We need 1,2,3,...
            stars = StarConversion(stars.split()[1])
            # Description
            try:
                description = driver.find_element_by_xpath("//*[@id='content_inner']/article/p")
            except:
                description = None
            # UPC ID
            upc = driver.find_element_by_xpath("//*[@id='content_inner']/article/table/tbody/tr[1]/td")
            # Tax imposed in the book
            tax = driver.find_element_by_xpath("//*[@id='content_inner']/article/table/tbody/tr[5]/td")
            # Category of the book
            category_a =  driver.find_element_by_xpath("//*[@id='default']/div/div/ul/li[3]/a")
            # Download book poster image
            try:
                # get book poster link
                poster_url = driver.find_element_by_xpath("//*[@id='product_gallery']/div/div/div/img").get_attribute('src')
                r = requests.get(poster_url)
                i = Image.open(BytesIO(r.content))
                image_file_format = poster_url.split('.')[-1]
                i.save(f'{folder_name}/{str(book_num)}_{title.text}.{image_file_format}')
            except:
                pass
            book_num += 1

            # Define a dictionary with details we need
            
            
            book = {
                "Title": title.text,
                "Category": category_a.text,
                "Stock": stock,
                "Stars": stars,
                "Price": price.text,
                "Tax": tax.text,
                "UPC": upc.text,
            }
            if description == None:
                book["Description"] = None
            else:
                book["Description"] = description.text
               
            # append book to all details
            all_details.append(book)
    except:
        # Lets just close the browser if we run to an error
        driver.close()

# save the information into a CSV file
df = pd.DataFrame(all_details)
df.to_csv("Books.csv", encoding='utf-8')

time.sleep(3)
driver.close()


Page link : http://books.toscrape.com/catalogue/category/books_1/page-1.html
Book num  :   1  2  3  4  5  6  7  8  9  10  11  12  13  14  15  16  17  18  19  20  
Page link : http://books.toscrape.com/catalogue/category/books_1/page-2.html
Book num  :   1  2  3  4  5  6  7  8  9  10  11  12  13  14  15  16  17  18  19  20  
Page link : http://books.toscrape.com/catalogue/category/books_1/page-3.html
Book num  :   1  2  3  4  5  6  7  8  9  10  11  12  13  14  15  16  17  18  19  20  
Page link : http://books.toscrape.com/catalogue/category/books_1/page-4.html
Book num  :   1  2  3  4  5  6  7  8  9  10  11  12  13  14  15  16  17  18  19  20  
Page link : http://books.toscrape.com/catalogue/category/books_1/page-5.html
Book num  :   1  2  3  4  5  6  7  8  9  10  11  12  13  14  15  16  17  18  19  20  
Page link : http://books.toscrape.com/catalogue/category/books_1/page-6.html
Book num  :   1  2  3  4  5  6  7  8  9  10  11  12  13  14  15  16  17  18  19  20  
Page link : http://bo